In [3]:

# only run this once if required and margin list has changes from website!!! 
# just run everything else below this cell on regular basis

# import margin from westpac website
import tabula
import pandas as pd
myfile = 'https://www.westpac.com.au/content/dam/public/wbc/documents/pdf/pb/1999030/WOIL_Acceptable_Securities_List.pdf'

df2 = tabula.read_pdf(myfile,pages='all')
frames = [df2[0],df2[1],df2[2],df2[3]]
df2 = pd.concat(frames)
df2.drop(['Unnamed: 0','Unnamed: 1','APIR Code','Fund Name','APIR Code Fund Name'],axis=1) 
split1 = df2[['ASX Code','Security Name','LVR']]
split2 = df2[['ASX Code.1','Security Name.1','LVR.1']]
split2.columns=['ASX Code','Security Name','LVR']
df2 = pd.concat([split1,split2])
df2 = df2.dropna()
df2 = df2.drop_duplicates(subset=['ASX Code'])
#df2.loc[:,'ASX Code']=df2['ASX Code'] + '.AX'


a = pd.read_csv('margin.csv')
df = pd.merge(a,df2,on='ASX Code',how='left')
df = df.drop(['LVR_x','Security Name_x'],axis=1)
df = df.rename(columns={'Security Name_y':'Security Name','LVR_y':'LVR'})
df = df[['ASX Code','Security Name','LVR','Industry','Valuation']]
#df = df.style.set_properties(**{'text-align': 'left'})

df.to_csv('marg.csv')

In [4]:
import pandas as pd
import datetime
import math
import numpy as np
import pandas_datareader.data as web
from pandas import Series, DataFrame
from sklearn import preprocessing
import matplotlib.pyplot as plt
from datetime import date, timedelta
import yfinance as yf

pd.set_option('mode.chained_assignment',None)    # turn off chain error warning since I use loc function properly
pd.options.display.float_format = '{:,.2f}'.format


# myfunction to use MACD to give signal


# 1. look at mac
# 2. look at overall market 

def MAC(MACDiff,MACD,Trigger,Movement):
    if MACDiff >= 0.01:
        if MACD < 0 and MACD < Trigger:
            action = 'Buy - accumulate'
        elif MACD < 0 and MACD > Trigger and Movement > 0:
            action = 'Buy - positive trigger'
        elif MACD < 0 and MACD > Trigger and Movement < 0:
            action = 'Buy - weak trigger'
        elif MACD > 0 and MACD > Trigger:
            action = 'Buy - positive signal'
        else:
            action = 'MACD increases but no comment'
    elif MACDiff > -0.01 and MACDiff < 0.01:
        if Movement > 0:
            action = 'MACD flat with buy activity'
        elif Movement < 0:
            action = 'MACD flat with sell activity'
        else:
            action = 'Sideway'
    else:    # MACDIFF IS NEGATIVE
        if MACD > 0:
            action = 'Buy - but momentum declines'    
        elif MACD < 0 and MACD < Trigger:
            action = 'Sell'
        elif MACD < 0 and Movement > 0:                 
            action = 'Sell - a bit of buy support'       #technical bad but ppl still buy? must be good value
        else:
            action = 'MACD declines'
    return action



# my purchase price table
myInput = pd.DataFrame({'Code':['AGL.AX','WPL.AX','WBC.AX'],
                        'PurchasePrice': [13.535,19.98,21.35],
                        'Unit': [4447,1275,163],
                        'InPortfolio':['y','y','y']
                       })

# pick up data from yahoo finance
start = datetime.datetime(2020, 7, 1)
end = datetime.date.today() + datetime.timedelta(days=0)
#end = datetime.datetime(2020,10,14)


# new code to pick up from csv instead with more details

mystock = pd.read_csv('margin.csv')
mystock = mystock['ASX Code']
#mystock = mystock.replace('^AORD.AX','^AORD')    # all ord does not come with .ax

# convert to string to make it better with dowloanding tracker
mystring = ''
mylist = []
for i in mystock:
    if i in ['^AORD','^DJI','^FTSE','CL=F','^AXJO','^IXIC','^HIS','^N225']:     # indexes no need to add .AX
        mystring = mystring + ' ' + i
        mylist.append(i)
    else:
        mystring = mystring + ' ' + i + '.AX'
        mylist.append(i+'.AX')

# download data 
data = yf.download(mystring, start=start, end=end, group_by="ticker")


# create columns for dataframe and the dataframe itself
mycol = ['Code','Close','Movement','Volume','ShortAvg','LongAvg','MACD','Trigger','Max','Min','2DAvg','VolDiff','P/Max']
mytbl = pd.DataFrame(columns=mycol)


# looping through the list to modify table
for i in mylist:
    #df = web.DataReader(i, 'yahoo', start, end)[['Close','Volume']]    # old code using datareader (not working well)
    #df = yf.download(i, start=start, end=end)[['Close','Volume']]      # old Yfinance code
    
    df = data[i][['Close','Volume']]     # new one using Yfinance
    df['Code'] = i
   #df.loc[:,'ShortAvg'] = df.Close.rolling(window=7).mean()
    df.loc[:,'ShortAvg'] = df.Close.ewm(span=5).mean()        
    df.loc[:,'LongAvg'] = df.Close.ewm(span=14).mean()
    df['MACD'] = df['ShortAvg'] - df['LongAvg']
    df['Trigger'] = df.MACD.rolling(window=4).mean()
    df['Movement'] = df['Close'].diff(1)
    df['2DAvg'] = df.Movement.rolling(window=2).sum()
    df['Max'] = df['Close'].max()
    df['Min'] = df['Close'].min()
    df['VolDiff'] = df['Volume'].diff(1)
    df['P/Max'] = df['Close']/df['Max']
    mytbl = mytbl.append(df)

mytbl = mytbl.reset_index()
mytbl = mytbl.rename(columns={'index':'Date'})


############## SQL TO MERGE WITH MY INPUT ##############
code = pd.merge(mytbl,myInput,on='Code',how='left')
code = code.rename(columns={'index':'Date'})


#remove null values
code['Unit'] = code['Unit'].fillna(0)
code['PurchasePrice'] = code['PurchasePrice'].fillna(0)
code['InPortfolio'] = code['InPortfolio'].fillna('n')

code['Profit'] = (code['Close'] - code['PurchasePrice']) * code['Unit']
#code['AvgPctg'] = (code['Wkly']-code['TwoWks'])/code['TwoWks'] * 100

#code['Diff'] = code['Wkly'] - code['TwoWks']
#code['DdownCnt']=code['Ddown'].rolling(window=14,min_periods=1).count()




# create another table to give signal
#mysignal = code
#mysignal['MACD_Diff'] = mysignal['MACD'].diff(1)
#mysignal['Advice']=mysignal.apply(lambda x:MAC(x['MACD_Diff'],x['MACD'],x['Trigger'],x['Movement']),axis=1)




code['MACD_Diff'] = code['MACD'].diff(1)
code['Advice']=code.apply(lambda x:MAC(x['MACD_Diff'],x['MACD'],x['Trigger'],x['2DAvg']),axis=1)


code.to_csv('YahooFinance.csv')


#code.head()






#code.to_csv('mysignal.csv')





[*********************100%***********************]  399 of 399 completed


In [5]:
import datetime as dt
start = datetime.date.today() + datetime.timedelta(days=-5)
start = start.strftime("%m/%d/%Y")
end = dt.datetime.today().strftime("%m/%d/%Y")
mydate = pd.date_range(start,end)



mycol = ['Date','Code','Close','MACD','Trigger','MACD_Diff','Movement','2DAvg','Min','Max','P/Max','Advice']
rec = code[mycol]

print ('******Overall Index Performance************')

rec[(rec['Code'].isin(['^AORD','WPL.AX','AGL.AX','IAG.AX'])) & (rec['Date']>start) & (rec['Date']<=end)].sort_values(by=['Code','Date'],ascending=False)

******Overall Index Performance************


,Date,Code,Close,MACD,Trigger,MACD_Diff,Movement,2DAvg,Min,Max,P/Max,Advice
769,2020-11-17,^AORD,nan,129.07,125.94,0.00,nan,nan,"5,973.50","6,687.00",nan,Sideway
768,2020-11-16,^AORD,"6,687.00",129.07,123.12,8.07,77.70,67.60,"5,973.50","6,687.00",1.00,Buy - positive signal
42569,2020-11-17,WPL.AX,21.55,1.07,0.97,0.11,0.71,0.94,16.80,21.77,0.99,Buy - positive signal
42568,2020-11-16,WPL.AX,20.84,0.96,0.89,0.02,0.23,0.07,16.80,21.77,0.96,Buy - positive signal
18589,2020-11-17,IAG.AX,5.41,0.19,0.18,0.01,0.10,0.18,4.38,5.83,0.93,Buy - positive signal
18588,2020-11-16,IAG.AX,5.31,0.18,0.18,0.00,0.08,0.00,4.38,5.83,0.91,Sideway
1979,2020-11-17,AGL.AX,12.89,-0.02,0.01,-0.03,-0.20,0.02,12.48,17.59,0.73,Sell
1978,2020-11-16,AGL.AX,13.09,0.01,0.03,0.01,0.22,-0.01,12.48,17.59,0.74,MACD increases but no comment


In [6]:
import numpy as np


mydate = datetime.date.today().strftime("%Y-%m-%d")

# the format of this input : Start, End, Code, Type, Unit, Price
# type --> b = buy , s = sell , w = watchlist
myval = [
    # Bought Date, Sold Date, Code, Type, Unit, Price
    ['2020-8-26',mydate,'WPL.AX','B',1275,19.98],
    ['2020-7-22',mydate,'WPL.AX','B',1263,20.89],
    ['2020-10-15',mydate,'AGL.AX','B',4447,13.535],
    ['2020-8-27',mydate,'IAG.AX','B',6060,4.955],
    ['2020-11-03',mydate,'WPL.AX','S',1120,18.37], 
    ['2020-11-05',mydate,'AGL.AX','W',1546,12.93], 
     ]

# create two tables (manual input vs table for ALLORD)
myinput = pd.DataFrame(myval,columns=['Date','Date_to','Code','Type','Unit','Price'])
myinput['Date']= myinput['Date'].astype('datetime64[ns]')                  # change from objec to the date type
allord = code[code['Code']=='^AORD'][['Date','Close']]



# merge tables to create a consolidated with share vs ordinary

benchmark = pd.merge(myinput,allord,on='Date',how='inner')
#benchmark['']

# loop all the description and create dataframe 
for i in range(len(benchmark['Date'])):
        StartDate = benchmark.loc[i,'Date']
        EndDate = benchmark.loc[i,'Date_to']
        Code = benchmark.loc[i,'Code']
        Unit = benchmark.loc[i,'Unit']
        Price = benchmark.loc[i,'Price']
        Index = benchmark.loc[i,'Close']
        Type = benchmark.loc[i,'Type']
        
        mydate = pd.date_range(StartDate,EndDate)
        mypurchasedate = np.repeat(StartDate,len(mydate))
        myunit = np.repeat(Unit,len(mydate))
        myprice = np.repeat(Price,len(mydate))
        mycode = Code
        
        if i ==0 : # create the main dataframe the first time
            df1 = pd.DataFrame ({'Date':mydate, 'Bought_Date':StartDate,'Type':Type,'Code':mycode,'Unit':myunit,'Price':myprice,'Index':Index})
        else:      # create tmp and append to main dataframe
            tmp = pd.DataFrame ({'Date':mydate, 'Bought_Date':StartDate,'Type':Type,'Code':mycode,'Unit':myunit,'Price':myprice,'Index':Index})
            df1 = df1.append(tmp)
            

            
# create final profit table         

mydate = datetime.date.today() + datetime.timedelta(days=0)    #change time delta accordingly, default = 0 or now
mydate = mydate.strftime("%Y-%m-%d")
myprofit =  pd.merge(df1,code,on=['Date','Code'],how='inner')[['Date','Bought_Date','Type','Code','Unit_x','Price','Close','Index']]
myprofit =  pd.merge(myprofit,allord,on='Date',how='inner')
myprofit = myprofit.rename(columns={'Unit_x':'Unit','Close_x':'Close','Close_y':'Index_Close'})
myprofit = myprofit.sort_values(by=['Type'])
myprofit['Index_Unit'] = (myprofit['Unit'] * myprofit['Price'])/myprofit['Index']
myprofit['MyProfit'] = (myprofit['Close'] - myprofit['Price']) * myprofit['Unit']
myprofit['Benchmark'] = (myprofit['Index_Close'] - myprofit['Index']) * myprofit['Index_Unit']
#myprofit['Profit_Impact'] = myprofit['MyProfit'] - myprofit['Benchmark']
myprofit['Profit%'] = (myprofit['Close']-myprofit['Price'])/myprofit['Price']
myprofit['Index%'] = (myprofit['Index_Close']-myprofit['Index'])/myprofit['Index_Close']
myprofit['Diff%'] = myprofit['Profit%'] - myprofit['Index%']  
#myprofit.style.format({'Profit%':"{:.2%}"})


print ('########### Benchmark against all ordinaries ##############')

mycolumns = ['Type','Code','Date','Bought_Date','Unit','Price','Close','MyProfit','Benchmark','Profit%','Index%','Diff%']
myprofit = myprofit[mycolumns]

#myprofit[myprofit['Date'] == mydate]
myprofit[myprofit['Date']==mydate].sort_values(by=['Type','Code','Bought_Date'])
#myprofit.style.format({'Profit%':"{:.2%}",'Index%':"{:.2%}"})
myprofit[myprofit['Date'] == mydate]


########### Benchmark against all ordinaries ##############


,Type,Code,Date,Bought_Date,Unit,Price,Close,MyProfit,Benchmark,Profit%,Index%,Diff%


In [630]:
code.tail(10)

,Date,Code,Close,Movement,Volume,ShortAvg,LongAvg,MACD,Trigger,Max,Min,2DAvg,VolDiff,P/Max,PurchasePrice,Unit,InPortfolio,Profit,MACD_Diff,Advice
41885,2020-10-28,ZYUS.AX,9.87,-0.26,"38,190.00",10.10,10.16,-0.06,0.01,10.49,9.69,-0.39,"26,108.00",0.94,0.00,0.00,n,0.00,-0.07,Sell
41886,2020-10-29,ZYUS.AX,9.78,-0.09,"12,346.00",10.00,10.11,-0.12,-0.03,10.49,9.69,-0.35,"-25,844.00",0.93,0.00,0.00,n,0.00,-0.06,Sell
41887,2020-10-30,ZYUS.AX,9.79,0.01,"37,182.00",9.93,10.07,-0.14,-0.08,10.49,9.69,-0.08,"24,836.00",0.93,0.00,0.00,n,0.00,-0.03,Sell
41888,2020-11-02,ZYUS.AX,9.92,0.13,"22,750.00",9.92,10.05,-0.12,-0.11,10.49,9.69,0.14,"-14,432.00",0.95,0.00,0.00,n,0.00,0.02,MACD flat with buy activity
41889,2020-11-03,ZYUS.AX,10.17,0.25,"8,009.00",10.01,10.06,-0.06,-0.11,10.49,9.69,0.38,"-14,741.00",0.97,0.00,0.00,n,0.00,0.07,Buy - positive trigger
41890,2020-11-04,ZYUS.AX,10.25,0.08,"295,265.00",10.09,10.09,-0.00,-0.08,10.49,9.69,0.33,"287,256.00",0.98,0.00,0.00,n,0.00,0.06,Buy - positive trigger
41891,2020-11-05,ZYUS.AX,9.91,-0.34,"43,018.00",10.03,10.07,-0.04,-0.05,10.49,9.69,-0.26,"-252,247.00",0.94,0.00,0.00,n,0.00,-0.04,MACD declines
41892,2020-11-06,ZYUS.AX,9.91,0.00,"14,924.00",9.99,10.04,-0.06,-0.04,10.49,9.69,-0.34,"-28,094.00",0.94,0.00,0.00,n,0.00,-0.02,MACD flat with sell activity
41893,2020-11-08,ZYUS.AX,nan,nan,nan,9.99,10.04,-0.06,-0.04,10.49,9.69,nan,nan,nan,0.00,0.00,n,nan,0.00,Sideway
41894,2020-11-09,ZYUS.AX,nan,nan,nan,9.99,10.04,-0.06,-0.05,10.49,9.69,nan,nan,nan,0.00,0.00,n,nan,0.00,Sideway


In [628]:
pd.options.display.float_format = '{:,.2f}'.format

myprofit = myprofit[myprofit['Date']=='2020-11-06'].sort_values(by=['Type','Code','Bought_Date'])

myprofit
#myprofit.style.format({'Profit%':"{:.2%}",'Index%':"{:.2%}"})

,Type,Code,Date,Bought_Date,Unit,Price,Close,MyProfit,Benchmark,Profit%,Index%
181,B,AGL.AX,2020-11-06,2020-10-15,4447,13.54,12.85,"-3,046.19",-180.17,-0.05,-0.00
183,B,IAG.AX,2020-11-06,2020-08-27,6060,4.96,4.85,-636.30,401.59,-0.02,0.01
179,B,WPL.AX,2020-11-06,2020-07-22,1263,20.89,18.19,"-3,410.10",862.34,-0.13,0.03
178,B,WPL.AX,2020-11-06,2020-08-26,1275,19.98,18.19,"-2,282.25",406.73,-0.09,0.02
180,S,WPL.AX,2020-11-06,2020-11-03,1120,18.37,18.19,-201.60,434.30,-0.01,0.02
182,W,AGL.AX,2020-11-06,2020-11-05,1546,12.93,12.85,-123.68,160.70,-0.01,0.01


In [291]:
 pd.date_range(start, periods=5, freq=pd.offsets.BDay())

DatetimeIndex(['2020-07-01', '2020-07-02', '2020-07-03', '2020-07-06',
               '2020-07-07'],
              dtype='datetime64[ns]', freq='B')

In [317]:
import datetime as dt
s = dt.datetime.today().strftime("%m/%d/%Y")

'11/02/2020'

In [318]:
import datetime as dt
start = datetime.datetime(2020, 7, 1)
end = datetime.date.today() + datetime.timedelta(days=1)

,StartDate,EndDate,Code,Type,Unit,Price
0,2020-10-27,2020-11-02,WPL.AX,B,1120,17.93
1,2020-10-15,2020-11-02,AGL.AX,B,4447,13.54
